In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService

import pandas as pd
import time
import re
import datetime
import numpy as np

D:\anaconda_nowa\envs\applied_mathematics\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
D:\anaconda_nowa\envs\applied_mathematics\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
D:\anaconda_nowa\envs\applied_mathematics\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


Analiza będzie przeprowadzona dla tych miast:

In [67]:
poland_cities = ["Białystok", "Bydgoszcz", "Gdańsk", "Gorzów Wielkopolski", "Katowice", "Kielce", "Kraków", "Lublin",
                 "Łódź", "Olsztyn", "Opole", "Poznań", "Rzeszów", "Szczecin", "Toruń", "Warszawa", "Wrocław",
                 "Zielona Góra"]
cities = ["Stambuł", "Londyn", "Berlin", "Madryt", "Rzym", "Bukareszt", "Paryż", "Wiedeń", "Hamburg", "Budapeszt", "Belgrad", "Barcelona", "Monachium", "Mediolan", "Sofia", "Praga", "Birmingham", "Lizbona"]
all_cities = poland_cities + cities

Będziemy sprawdzać dla:
- 1 dorosłego,
- 2 dorosłych,
- 7 dorosłych

I dla:
- 1 dnia
- 2 dni
- 7 dni
- 14 dni

In [74]:
days = (1, 2, 7, 14)
persons = (2, 7) # 1 ad, 2 ad, 7 ad

In [75]:
# dataframe = pd.DataFrame({"City":[], "hotel_name":[], "people_number":[], "days number":[], "Price":[], "amenities":[], "guest_opinions":[], "average_opinion":[]})

In [76]:
for i in days:
    for j in persons:
        for k in all_cities:
            webOptions = webdriver.ChromeOptions()
            # webOptions.add_argument("--headless")
            webOptions.add_argument("--window-size=1920,1080")
            webOptions.add_argument("–disable-gpu")
            driver = webdriver.Chrome(service=ChromeService(executable_path=ChromeDriverManager().install()), options=webOptions)
            driver.maximize_window()
            driver.get("https://www.booking.com/")
            # driver.minimize_window()
            time.sleep(2)
            accept = driver.find_element(By.XPATH, "//button[@id='onetrust-accept-btn-handler']")
            accept.click()
            city = driver.find_element(By.XPATH, "//input[@class='eb46370fe1']")
            city.send_keys(k)
            time.sleep(2)
            date = driver.find_element(By.XPATH, "//button[@data-testid='date-display-field-start']")
            date.click()
            time.sleep(2)
            data_dzisiejsza = driver.find_element(By.XPATH, f"//span[@data-date='{datetime.date.today().strftime('%Y-%m-%d')}']")
            data_dzisiejsza.click()
            data_jutrzejsza = driver.find_element(By.XPATH, f"//span[@data-date='{(datetime.date.today() + datetime.timedelta(i)).strftime('%Y-%m-%d')}']")
            data_jutrzejsza.click()
            time.sleep(1)
            if j != 2:
                persons = driver.find_element(By.XPATH, "//button[@class='a83ed08757 ebbedaf8ac ada2387af8']")
                persons.click()
                time.sleep(1)
                if j == 1:
                    one = driver.find_element(By.XPATH, "//button[@class='a83ed08757 c21c56c305 f38b6daa18 d691166b09 ab98298258 deab83296e bb803d8689 e91c91fa93']")
                    one.click()
                elif j == 7:
                    seven = driver.find_element(By.XPATH, "//button[@class='a83ed08757 c21c56c305 f38b6daa18 d691166b09 ab98298258 deab83296e bb803d8689 f4d78af12a']")
                    for _ in range(5):
                        seven.click()
            search = driver.find_element(By.XPATH, "//button[@class='a83ed08757 c21c56c305 a4c1805887 f671049264 d2529514af c082d89982 cceeb8986b']")
            search.click()
            time.sleep(5)
            try:
                cancel = driver.find_element(By.XPATH, "//button[@class='a83ed08757 c21c56c305 f38b6daa18 d691166b09 ab98298258 deab83296e f4552b6561' and @aria-label='Zamknij okno logowania.']")
                cancel.click()
            except:
                pass
            elements = list(driver.find_elements(By.XPATH, "//div[@class='f6431b446c a15b38c233']"))
            opinions = [x.text for x in driver.find_elements(By.XPATH, "//div[@class='a3b8729ab1 d86cee9b25']")]
            prices = [x.text for x in driver.find_elements(By.XPATH, "//span[@class='f6431b446c fbfd7c1165 e84eb96b1f']")]
            if len(elements) != len(opinions):
                booking_news = driver.find_elements(By.XPATH, "//div[@class='c82435a4b8 a178069f51 a6ae3c2b40 a18aeea94d d794b7a0f7 f53e278e95 c6710787a4' or @class='c82435a4b8 a178069f51 a6ae3c2b40 a18aeea94d d794b7a0f7 f53e278e95 c6710787a4 bb369fb627']")
                wrong_data = np.array(["Nowość na Booking.com" in booking_news[c].text and "opinii" not in booking_news[c].text for c in range(len(booking_news))])
            w = 0
            for l in range(len(elements)):
                if len(elements) != len(opinions):
                    if wrong_data[l] == True:
                        w += 1
                        continue
                try:
                    elements[l].click()
                    hotel_name = elements[l].text
                    time.sleep(10)
                    driver.switch_to.window(driver.window_handles[1])
                    udogodnienia = driver.find_element(By.XPATH, "//section[@class='propertySectionFixedSpacing e1a6df845d' and @id='hp_facilities_box']")
                    opinie_gosci = driver.find_element(By.XPATH, "//div[@data-testid='PropertyReviewsRegionBlock']")
                    dataframe.loc[len(dataframe)] = [k, hotel_name, j, i, prices[l], udogodnienia.text, opinie_gosci.text, opinions[l-w]]
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
                except:
                    pass
            driver.quit()

In [151]:
webOptions = webdriver.ChromeOptions()
# webOptions.add_argument("--headless")
webOptions.add_argument("--window-size=1920,1080")
webOptions.add_argument("–disable-gpu")
driver = webdriver.Chrome(service=ChromeService(executable_path=ChromeDriverManager().install()), options=webOptions)
driver.maximize_window()
driver.get("https://www.booking.com/")
# driver.minimize_window()
time.sleep(2)

In [152]:
accept = driver.find_element(By.XPATH, "//button[@id='onetrust-accept-btn-handler']")
accept.click()
city = driver.find_element(By.XPATH, "//input[@class='eb46370fe1']")
city.send_keys("Gorzów Wielkopolski")
time.sleep(2)
date = driver.find_element(By.XPATH, "//button[@data-testid='date-display-field-start']")
date.click()
time.sleep(2)
data_dzisiejsza = driver.find_element(By.XPATH, f"//span[@data-date='{datetime.date.today().strftime('%Y-%m-%d')}']")
data_dzisiejsza.click()
data_jutrzejsza = driver.find_element(By.XPATH, f"//span[@data-date='{(datetime.date.today() + datetime.timedelta(1)).strftime('%Y-%m-%d')}']")
data_jutrzejsza.click()
time.sleep(1)
search = driver.find_element(By.XPATH, "//button[@class='a83ed08757 c21c56c305 a4c1805887 f671049264 d2529514af c082d89982 cceeb8986b']")
search.click()

In [153]:
time.sleep(3)
cancel = driver.find_element(By.XPATH, "//button[@class='a83ed08757 c21c56c305 f38b6daa18 d691166b09 ab98298258 deab83296e f4552b6561' and @aria-label='Zamknij okno logowania.']")
cancel.click()

In [154]:
elements = driver.find_elements(By.XPATH, "//div[@class='f6431b446c a15b38c233']")

In [156]:
opinions = [x.text for x in driver.find_elements(By.XPATH, "//div[@class='a3b8729ab1 d86cee9b25']")]
prices = [x.text for x in driver.find_elements(By.XPATH, "//span[@class='f6431b446c fbfd7c1165 e84eb96b1f']")]

In [177]:
a = driver.find_elements(By.XPATH, "//div[@class='c82435a4b8 a178069f51 a6ae3c2b40 a18aeea94d d794b7a0f7 f53e278e95 c6710787a4']")

In [178]:
b = np.array(["Nowość na Booking.com" in a[i].text and "opinii" not in a[i].text for i in range(len(a))])

In [179]:
list(np.where(b == True)[0])

[9]

In [83]:
elements[1].click()
time.sleep(2)
driver.switch_to.window(driver.window_handles[1])

In [84]:
udogodnienia = driver.find_element(By.XPATH, "//section[@class='propertySectionFixedSpacing e1a6df845d' and @id='hp_facilities_box']")

In [85]:
udogodnienia.text

'Udogodnienia w obiekcie MirrorRoom PL Downtown\nZobacz dostępność\nNajpopularniejsze udogodnienia\npokoje dla niepalących\nSzybkie bezpłatne Wi-Fi (555 Mb/s)\nogród\nŁazienka\npapier toaletowy\nbezpłatny zestaw kosmetyków\nwspólna łazienka\nSypialnia\npościel\nszafa lub garderoba\nWidok\nwidok na miasto\nwidok\nOkolica\nogród\nKuchnia\nwspólna kuchnia\nstół\nśrodki czystości\npłyta kuchenna\npiekarnik\nprzybory kuchenne\nczajnik elektryczny\nkuchenka mikrofalowa\nlodówka\nSalon\nbiurko\nInternet\nSzybkie bezpłatne Wi-Fi 555 Mb/s. Odpowiednie do streamingu treści w jakości 4K i prowadzenia wideorozmów na wielu urządzeniach. Test prędkości wykonany przez gospodarza.\nParking\nParking nie jest dostępny.\nUsługi\nekspresowe zameldowanie / wymeldowanie\nBezpieczeństwo\nmonitoring w pomieszczeniach ogólnodostępnych\nOgólne\ncałkowity zakaz palenia\nogrzewanie\npokoje dla niepalących\nJęzyki\nangielski'

In [86]:
opinie_gosci = driver.find_element(By.XPATH, "//div[@data-testid='PropertyReviewsRegionBlock']")

In [87]:
opinie_gosci.text

'Opinie Gości\nZobacz dostępność\n6,3\nPrzyjemny · 578 opinii\nPrzeczytaj wszystkie opinie\nKategorie:\nPersonel\n7,0\nUdogodnienia\n6,4\nCzystość\n6,5\nKomfort\n6,6\nStosunek jakości do ceny\n6,8\nLokalizacja\n8,1\nBezpłatne WiFi\n8,2\nNiska ocena Warszawa\nWybierz tematy, aby przeczytać opinie:\nPokój\nLokalizacja\nCzystość\nŁóżko\nRęcznik\nPrzeczytaj wszystkie opinie'

In [80]:
driver.close()

In [59]:
driver.quit()